# Finetuning

Instead of doing RAG, in this notebook, let's actually re-train the model

## Downloading the Model

If you have not done so, please download the model via the follwoing cell

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
token = os.getenv('HF_TOKEN')
os.environ['HF_TOKEN'] = token
!huggingface-cli login --token $HF_TOKEN
!wget -P ..models/Llama-2-7b-chat/ https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_K_M.gguf

## The model task before fine-tuning

In [1]:
from llama_cpp import Llama

In [2]:

llm = Llama(model_path="../models/llama-2-7b-chat.Q4_K_M.gguf", n_ctx=2048, verbose=False)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../models/llama-2-7b-chat.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32      

In [3]:
from IPython.display import clear_output, display, Markdown

class Llama2_chat:

    def __init__(self, system_message):
        """Initializes the Chat with the system message."""
        self._messages = []
        self._messages = self.append_system_message(self._messages, content=system_message)

    def append_message(self, messages, role, content):
        messages.append({"role": role, "content": content})
        return messages

    def append_system_message(self, messages, content):
        self.append_message(messages, "system", content)
        return messages

    def append_user_message(self, messages, content):
        self.append_message(messages, "user", content)
        return messages

    def append_assistant_message(self, messages, content):
        self.append_message(messages, "assistant", content)
        return messages

    def append_assistant_stream(self, messages, content):
        if self._messages[-1]['role'] != "assistant":
            self.append_message(messages, "assistant", content)
        else:
            self._messages[-1]['content'] += content
        return messages

    def get_llama2_response(self, messages, **kwargs):
        self.model_response = llm.create_chat_completion(messages = messages, **kwargs)
        return self.model_response['choices'][0]['message']['content']

    def get_llama2_response_stream(self, messages, **kwargs):
        self.model_response = llm.create_chat_completion(messages = messages, stream=True, **kwargs)
        return self.model_response

    def format_markdown_with_style(self, text, font_size=16):
        """
        Wraps the given text in a <span> tag with the specified font size.

        Parameters:
        text (str): The Markdown text to be formatted.
        font_size (int, optional): The font size to apply. Defaults to 16.

        Returns:
        str: The formatted Markdown string with HTML styling.
        """
        return f"<span style='font-size: {font_size}px;'>{text}</span>"

    def prompt_llama2(self, user_prompt):
        self._messages = self.append_user_message(self._messages, content=user_prompt)
        llama2_response = self.get_llama2_response(self._messages)
        self._messages = self.append_assistant_message(self._messages, content=llama2_response)
        display(Markdown(self.format_markdown_with_style(llama2_response)))

    def prompt_llama2_stream(self, user_prompt):
        self._messages = self.append_user_message(self._messages, content=user_prompt)
        llama2_response_stream = self.get_llama2_response_stream(self._messages)
        
        complete_stream = ""
        display_id = 'llama2_stream_display'  # Unique display ID

        for part in llama2_response_stream:
            # Check if 'content' key exists in the 'delta' dictionary
            if 'content' in part['choices'][0]['delta']:
                stream_content = part['choices'][0]['delta']['content']
                complete_stream += stream_content
                self._messages = self.append_assistant_stream(self._messages, content=stream_content)

                # Clear previous output and display new content
                clear_output(wait=True)
                display(Markdown(self.format_markdown_with_style(complete_stream)))

            else:
                # Handle the case where 'content' key is not present
                # For example, you can print a placeholder or do nothing
                # print("(no content)", end='')
                pass


In [4]:
chat = Llama2_chat("Answer in a very concise and accurate way")

In [6]:
#chat.prompt_llama2("Name the planets in the solar system")
#chat.prompt_llama2_stream("Please describe the route of the world trip for Wittmann Tours")
#chat.prompt_llama2_stream("What do you know about the Wittmann Tours world trip and its itinerary?")
#chat = Llama2_chat("Answering the context of the blog wittmann-tours.de. Be truthful, do not make things up. If you no not know, just be honest about it.")
#chat.prompt_llama2_stream("What do you know about the Wittmann Tours world trip and its itinerary?")
chat.prompt_llama2_stream("What do you know about wittmann-tours.de?")

<span style='font-size: 16px;'>  I'm just an AI and do not have access to specific information about Wittmann-Tours.de unless it is publicly available on the internet. However, I can suggest some possible ways to find out more about the website:
1. Check the website's domain registration information: You can use a whois lookup tool to see when the domain was registered, who the registrant is, and other details. This information may not be up-to-date or accurate, but it can give you an idea of when the website was created and who is responsible for it.
2. Look for reviews or testimonials: Check review websites like TripAdvisor, Google Reviews, or Yelp to see what other customers have to say about Wittmann-Tours.de. Be cautious of fake reviews, and look for patterns in the feedback.
3. Check the website's social media accounts: Look for the website's presence on social media platforms like Facebook, Twitter, or Instagram. This can give you an idea of how active the website is and what kind of content they post.
4. Search for news articles or press releases: You can use a search engine to look for news articles or press releases about Wittmann-Tours.de. This may provide information about the company's history, mission, and any notable events or achievements.
5. Contact the website directly: If you have specific questions about Wittmann-Tours.de, you can try contacting them directly through their website or by phone. They may be able to provide you with more detailed information about their services and policies.
It's important to note that some websites may not have a lot of publicly available information, especially if they are small or new. In these cases, it may be best to wait and see if more information becomes available over time.</span>

The model obviously does not know anything about wittmann-tours.de

## Preparing the Data

The goal is just to show the model additional data. Therefore, I follow the approach as layed out in [this tutorial](https://blog.gopenai.com/how-to-fine-tune-llama-2-on-mac-studio-4b42f317c975):



In [10]:
import os
import glob

path_to_blog = "/Users/chrwittm/Library/CloudStorage/OneDrive-Personal/Dokumente/GitRepos/clones/wordpress-to-markdown/out"

def get_blog_post_files(path):
    # Create a pattern to match all .mdx files in the directories under the base path
    pattern = os.path.join(path_to_blog, "**/*.mdx")

    # Use glob to find all files matching the pattern
    # The '**' pattern means "this directory and all subdirectories, recursively"
    # The '*.mdx' pattern means "all files ending with .mdx"
    file_list = glob.glob(pattern, recursive=True)

    # file_list now contains the full paths of all .mdx files
    return file_list

def get_blog_post(path):
    # Open the file in read mode
    with open(path, 'r') as file:
        # Read the contents of the file
        content = file.read()
    return content


files = get_blog_post_files(path_to_blog)
get_blog_post(files[0])[:1000]

    

'---\ntitle: \'Ritt auf Paso Peruanos im Colca Tal\'\ndescription: ""\npublished: 2018-11-05\nredirect_from: \n            - https://wittmann-tours.de/ritt-auf-paso-peruanos-im-colca-tal/\ncategories: "Colca, Colca Canyon, Colca Tal, Inka, Paso, Paso Peruano, Peru, Peru, Pferde, Reiten, Viscacha"\nhero: ../../../defaultHero.jpg\n---\nIn Peru ist man zu Recht sehr stolz auf die nationale Pferderasse des Landes, die [Paso Peruanos](https://de.wikipedia.org/wiki/Paso_Peruano). Ihre Besonderheit ist, dass sie eine spezielle, überaus bequeme Gangart haben, den [Paso Llano](https://de.wikipedia.org/wiki/Paso_Peruano#Gangmechanik), ähnlich dem Tölt der Islandpferde. Das Zuchtziel ("[Brio](https://de.wikipedia.org/wiki/Paso_Peruano#Interieur)") wird folgendermaßen definiert: "Eifrige Bereitwilligkeit kombiniert mit energischem Einsatz und ausdrucksvoller Präsentation". Auf diesen Prachtpferden wollten wir gerne reiten und zwar in der herrlichen Landschaft des Colca-Tales.\n\n![Colca-Panorama m

In [11]:
def add_starting_token(content):
    # Add the starting token <s> to the beginning of the content
    return "<s>" + content

add_starting_token(get_blog_post(files[0])[:1000])

'<s>---\ntitle: \'Ritt auf Paso Peruanos im Colca Tal\'\ndescription: ""\npublished: 2018-11-05\nredirect_from: \n            - https://wittmann-tours.de/ritt-auf-paso-peruanos-im-colca-tal/\ncategories: "Colca, Colca Canyon, Colca Tal, Inka, Paso, Paso Peruano, Peru, Peru, Pferde, Reiten, Viscacha"\nhero: ../../../defaultHero.jpg\n---\nIn Peru ist man zu Recht sehr stolz auf die nationale Pferderasse des Landes, die [Paso Peruanos](https://de.wikipedia.org/wiki/Paso_Peruano). Ihre Besonderheit ist, dass sie eine spezielle, überaus bequeme Gangart haben, den [Paso Llano](https://de.wikipedia.org/wiki/Paso_Peruano#Gangmechanik), ähnlich dem Tölt der Islandpferde. Das Zuchtziel ("[Brio](https://de.wikipedia.org/wiki/Paso_Peruano#Interieur)") wird folgendermaßen definiert: "Eifrige Bereitwilligkeit kombiniert mit energischem Einsatz und ausdrucksvoller Präsentation". Auf diesen Prachtpferden wollten wir gerne reiten und zwar in der herrlichen Landschaft des Colca-Tales.\n\n![Colca-Panoram

In [12]:
!pwd

/Users/chrwittm/Library/CloudStorage/OneDrive-Personal/Dokumente/GitRepos/lm-hackers


In [13]:
def combine_blog_posts_raw(base_path, output_file_path):
    # Get a list of all blog post files
    blog_post_files = get_blog_post_files(base_path)

    # Open the output file in write mode
    with open(output_file_path, 'w') as output_file:
        # Iterate over each blog post file
        for file_path in blog_post_files:
            # Get the content of the blog post
            content = get_blog_post(file_path)

            # Write the modified content to the output file
            output_file.write(content + "\n\n")  # Add a newline for separation between posts

# Example usage
output_path = "all-wittmann-tours-blog-posts-raw.txt"
combine_blog_posts_raw(path_to_blog, output_path)

In [14]:
def combine_blog_posts(base_path, output_file_path):
    # Get a list of all blog post files
    blog_post_files = get_blog_post_files(base_path)

    # Open the output file in write mode
    with open(output_file_path, 'w') as output_file:
        # Iterate over each blog post file
        for file_path in blog_post_files:
            # Get the content of the blog post
            content = get_blog_post(file_path)

            # Add the starting token to the content
            modified_content = add_starting_token(content)

            # Write the modified content to the output file
            output_file.write(modified_content + "\n\n")  # Add a newline for separation between posts

# Example usage
output_path = "all-wittmann-tours-blog-posts.txt"
combine_blog_posts(path_to_blog, output_path)


In [19]:
def shorten_image_links(content):
    # Regular expression to match the specific format of the image URLs
    url_pattern = r'http://wittmann-tours\.de/wp-content/uploads/(\d{4}/\d{2}/CW-\d{8}-\d{6})-[^.]+\.jpg'

    # Function to construct the shortened URL
    def url_replacer(match):
        return f"http://wittmann-tours.de/{match.group(1)}.jpg"

    # Replace all occurrences of the image URLs with the shortened version
    modified_content = re.sub(url_pattern, url_replacer, content)
    return modified_content

# Example usage
content = "Check out this image: http://wittmann-tours.de/wp-content/uploads/2018/10/CW-20180514-150742-0612-HDR-1024x683.jpg"
modified_content = shorten_image_links(content)
print(modified_content)

Check out this image: http://wittmann-tours.de/2018/10/CW-20180514-150742.jpg


In [26]:
import re

def insert_token_after_image_link(content, token="<s>"):
    # Regular expression to match markdown image links
    image_link_pattern = r'!\[.*?\]\(.*?\)'

    # Function to add the token after the image link
    def add_token(match):
        return match.group(0) + '\n' + token

    # Replace the image links with the image link followed by the token
    modified_content = re.sub(image_link_pattern, add_token, content)
    return modified_content

# Example usage
content = "Here is an image: ![Die Reittour im Colca-Tal beginnt](http://wittmann-tours.de/2018/10/CW-20180514-132856.jpg)"
modified_content = insert_token_after_image_link(content)
print(modified_content)


Here is an image: ![Die Reittour im Colca-Tal beginnt](http://wittmann-tours.de/2018/10/CW-20180514-132856.jpg)
<s>


In [27]:
def add_starting_token_to_blog_post(content):
    # Split the content into lines
    lines = content.split('\n')

    # Add "<s>" at the beginning of the content
    lines.insert(0, "<s>")

    # Iterate through lines and add "<s>" before each level 2 heading
    for i, line in enumerate(lines):
        if line.startswith("##"):
            lines[i] = "<s>\n" + line

    # Combine the lines back into a single string
    modified_content = '\n'.join(lines)
    return modified_content

import re

#def replace_image_links(content, replacement_token="<foto>\n<s>"):
#    # Regular expression to match typical image URLs
#    image_url_pattern = r'\(http[s]?://[^\s]*\.(jpg|jpeg|png|gif)\)'
#    
#    # Replace all occurrences of the image URLs with the replacement token
#    modified_content = re.sub(image_url_pattern, replacement_token, content)
#    return modified_content

def replace_image_links(content, replacement_token="<foto>\n<s>"):
    # Regular expression to match typical image URLs and local image paths
    image_url_pattern = r'\(http[s]?://[^\s]*\.(jpg|jpeg|png|gif)\)|\./img/wp-content-uploads-[^\s]*\.(jpg|jpeg|png|gif)'

    # Replace all occurrences of the image URLs and local image paths with the replacement token
    modified_content = re.sub(image_url_pattern, replacement_token, content)
    return modified_content

# Example usage
# content = "Here is a remote image (http://example.com/image.jpg) and a local image (./img/wp-content-uploads-2019-03-image.jpg)"
# modified_content = replace_image_links(content)
# print(modified_content)

def replace_links(content, replacement_token="<link>"):
    # Regular expression to match typical URLs
    url_pattern = r'\bhttp[s]?://[^\s]+\b'
    
    # Replace all occurrences of URLs with the replacement token
    modified_content = re.sub(url_pattern, replacement_token, content)
    return modified_content

# Example usage
# content = "Check out this website: http://example.com"
# modified_content = replace_links(content)
# print(modified_content)

def replace_text(content, text_to_replace, replacement_text):
    # Replace all occurrences of text_to_replace with replacement_text
    modified_content = content.replace(text_to_replace, replacement_text)
    return modified_content

# Example usage
# content = "This is a test --- to see if --- works."
# modified_content = replace_text(content, "---", "<s>")
# print(modified_content)

def cleanup_consecutive_start_tokens(content):
    # Regular expression to match consecutive <s> tokens separated by whitespace or line feeds
    pattern = r'(<s>\s*)+'

    # Replace matched patterns with a single <s> token
    cleaned_content = re.sub(pattern, '<s>\n', content)
    return cleaned_content

# Example usage
# content = "<s>\n\n<s>\n   <s>\n<s>"
# cleaned_content = cleanup_consecutive_start_tokens(content)
# print(cleaned_content)



def combine_blog_posts(file_list, combined_file_path):
    with open(combined_file_path, 'w') as combined_file:
        for file_path in file_list:
            # Get the content of each blog post
            content = get_blog_post(file_path)
            # Replace photos with token
            #content = replace_image_links(content) 
            # Replace links with token
            #content = replace_links(content) 
            # Replace special cases

            #shorten links
            content = shorten_image_links(content)
            content = insert_token_after_image_link(content)
            content = replace_text(content, "---", "<s>")
            content = replace_text(content, "<!--more-->", "<s>")
            content = replace_text(content, "<s>)", "<s>")
            content = replace_text(content, "![]()", "")
            content = replace_text(content, "hero: ../../../defaultHero.jpg", "")
            content = replace_text(content, 'description: ""', "")
            content = replace_text(content, "title: '", "# ")
            content = replace_text(content, "'\n", "\n")
            content = replace_text(content, "\n \n", "\n")
            

            # Add the starting token to the blog post
            modified_content = add_starting_token_to_blog_post(content)

            modified_content = cleanup_consecutive_start_tokens(modified_content)

            # Write the modified content to the combined file
            combined_file.write(modified_content + '\n\n')

# Example usage
file_list = get_blog_post_files(path_to_blog)
combine_blog_posts(file_list, 'all-wittmann-tours-blog-posts3.txt')


In [42]:
#chat = Llama2_chat("Answering the context of the blog wittmann-tours.de. Be truthful, do not make things up. If you no not know, just be honest about it.")
#chat = Llama2_chat("Answer in a very concise and accurate way")
#chat = Llama2_chat("Antworte kurz und prägnant")
#chat.prompt_llama2_stream("What do you know about the Wittmann Tours world trip and its itinerary?")
#chat.prompt_llama2_stream("What do you know about wittmann-tours.de?")
#chat.prompt_llama2_stream("Was kannst Du mir über wittmann-tours.de erzählen?")
#chat.prompt_llama2_stream("Tell me about fruit juices in Brazil")

It failed on the question "What do you know about the Wittmann Tours world trip and its itinerary?"- why? Base model?

"The Wittmann Tours World Trip is a once-in-a-lifetime adventure that takes travelers on an epic journey across six continents in just 12 months. Here's a brief overview of the itinerary:

Africa: Start the trip with a safari in South Africa, followed by visits to Namibia, Botswana, Zambia, and Kenya.

Asia: Travel through India, Nepal, Bhutan, Myanmar, Vietnam, Cambodia, Thailand, Indonesia, and Japan.

Australia & Oceania: Explore Australia's iconic landmarks, including Uluru (Ayers Rock), the Great Barrier Reef, and Sydney Harbour Bridge. Visit New Zealand for its stunning fjords, glaciers, and geothermal wonders.

South America: Journey through Brazil's Amazon rainforest, visit the Galapagos Islands in Ecuador, explore Peru's Inca Trail and Machu Picchu, and experience Argentina's Patagonian glaciers and Iguazu Falls.

Europe: Visit the world-famous cities of Paris, Rome, Barcelona, and London, as well as lesser-known gems like Croatia, Albania, and Bosnia & Herzegovina.

North America: Explore the Grand Canyon in Arizona, Yellowstone National Park in Wyoming, and the Rocky Mountains in Canada.

The itinerary includes a mix of guided tours, cultural experiences, and free time for independent exploration. The trip also offers opportunities to participate in local activities like hiking, biking, rafting, and wildlife encounters."

> 💡 **Practical tip** for fine-tuning: On a Mac, make sure your power settings to not reduce the speed when the screen is locked - most likely you do not want to watch the whole process 😉. On Sonoma, go to `System Settings > Battery > Options...` and activate the setting `Prevent automatic sleeping on power adapter when the display is off`. This way your Mac does not slow down when you are away from the keyboard.

## Process of failure

### Iteration 1 - Following tutorial of children story

Training the same way as in the tutorial used too much RAM, and the context length was too long. Therefore, I did the following

- Adjust the training data to reduce the number of tokens to fit it into 512/256
- let it run for 18h

```bash
#./finetune --model-base ./llama-2-7b-chat.Q4_K_M.gguf --train-data all-wittmann-tours-blog-posts1.txt --threads 6 --sample-start "<s>" --ctx 4096
#./finetune --model-base ./llama-2-7b-chat.Q4_K_M.gguf --train-data all-wittmann-tours-blog-posts1.txt --threads 6 --sample-start "<s>" --ctx 512
#./finetune --model-base ./llama-2-7b-chat.Q4_K_M.gguf --train-data all-wittmann-tours-blog-posts2.txt --threads 10 --sample-start "<s>" --ctx 256 --adam-iter 10 
```

After all optimizations with reducing tokens, this is the way to go:

`./finetune --model-base ./llama-2-7b-chat.Q4_K_M.gguf --train-data all-wittmann-tours-blog-posts2.txt --threads 10 --sample-start "<s>" --ctx 512 --adam-iter 319`

tokenize_file: warning: found 98 samples (max length 1217) that exceed context length of 512. samples will be cut off.
tokenize_file: warning: found 3086 samples (min length 3) that are shorter than context length of 512.
tokenize_file: total number of samples: 3184


tokenize_file: warning: found 8 samples (max length 749) that exceed context length of 512. samples will be cut off.
tokenize_file: warning: found 3104 samples (min length 4) that are shorter than context length of 512.
tokenize_file: total number of samples: 3112

`./finetune --model-base ./llama-2-7b-chat.Q4_K_M.gguf --train-data all-wittmann-tours-blog-posts2.txt --threads 10 --sample-start "<s>" --ctx 512 --adam-iter 312`

https://medium.com/@yukiarimo/unlock-the-full-potential-of-your-macbook-with-llama-cpp-run-and-fine-tune-llms-locally-0881f6b9c08d

The result was not good, and it kept saying that I need a base model similar to this:

```bash
./main --model ./models/llama-2-7b/ggml-model-q4_0.gguf
       --lora ggml-lora-LATEST-f32.gguf
       --lora-base ./models/llama-2-7b/ggml-model-f16.gguf
       --prompt "Can you please write a children's story with 200 words about father and son and friendship and bravery?"
```

Here is the way to create a base model:

How to download: https://llama-2.ai/getting-started-with-llama-2/
How to convert: https://www.secondstate.io/articles/convert-pytorch-to-gguf/

```bash
mkdir models
cd models
wget https://raw.githubusercontent.com/facebookresearch/llama/main/download.sh
bash download.sh
# - enter link from e-mail
# - enter 7B-chat

pip install gguf

python convert.py models/Llama-2-7b-chat/
```

In [40]:
#1st iteration
llm = Llama(model_path="llama-2-7b-chat.Q4_K_M.gguf",
            lora_path="./finetune1/ggml-lora-LATEST-f32.gguf",
            lora_base="models/Llama-2-7b-chat/ggml-model-f16.gguf",
            n_ctx=2048, verbose=False)

chat = Llama2_chat("Answer in a very concise and accurate way")
chat.prompt_llama2_stream("What do you know about wittmann-tours.de?")

<span style='font-size: 16px;'>  Wittmann Tours is a German tour operator that specializes in organizing tours to various destinations around the world, including Europe, Asia, Africa, and South America. They offer a wide range of travel services such as group tours, private tours, and tailor-made itineraries for individuals or small groups. Their website provides information on their tour packages, destinations, pricing, and contact details.</span>

### Iteration 2 - follow the docs

In the docs, there is no start token:
https://github.com/ggerganov/llama.cpp/blob/master/examples/finetune/README.md

```bash
./finetune \
        --model-base ./llama-2-7b-chat.Q4_K_M.gguf \
        --lora-out lora-llama-wittmann-tours-ITERATION.bin \
        --train-data "all-wittmann-tours-blog-posts-raw.txt" \
        --save-every 100 \
        --threads 10 --adam-iter 2160 --batch 4 --ctx 64 \
        --use-checkpointing
```

In [41]:
#2nd iteration
llm = Llama(model_path="llama-2-7b-chat.Q4_K_M.gguf",
            lora_path="./finetune2/lora-llama-wittmann-tours-LATEST.bin",
            lora_base="models/Llama-2-7b-chat/ggml-model-f16.gguf",
            n_ctx=2048, verbose=False)

chat = Llama2_chat("Answer in a very concise and accurate way")
chat.prompt_llama2_stream("What do you know about wittmann-tours.de?")
# This iteration seems to produce the best results, but it is not really good. Try the full load on a GPU

<span style='font-size: 16px;'>  Wittmann-Tours ist ein Reiseblog, der sich auf die Themen "Reisen" und "Kulinarisch" fokussiert. Er wurde von Christian Wittmann gegründet und seit 2014 wird er aktuell (Stand: 2020) gepflegt. Der Blog ist international orientiert und berichtet über Reisegeschichten, Tipps, Tricks, Eindrücke und Gerichte aus vielen Ländern der Welt. Die meisten Beiträge sind in deutscher Sprache geschrieben, aber auch einige in Englisch.

Wittmann-Tours ist ein sehr informativierter Blog, der sich auf die Themen "Kulinarisch" fokussiert. Christian Wittmann berichtet über seine eigenen Reiseregeln, Tipps für den Reiseanfang, die besten Gerichte und Lokalitäten, die er gefunden hat, und er gibt auch einige Tipps für die Reisenden mit sich.

Der Blog ist sehr lesenswert und bietet eine gute Grundlage für Reiseroute-Entscheidungen, da Christian Wittmann viele Erfahrungen gesammelt hat und auf vielen Seiten die besten Länder und Orte beschrieben wird. Die Beiträge sind sehr detailliert und informativ, aber auch sehr lesenswert und unterhaltsam.

Insgesamt ist Wittmann-Tours ein sehr nützlicher Blog für Reisende, die gerne gute Tipps und Eindrücke aus der Welt der Reise bekommen möchten.</span>

the results were a lot better, but I only trained for 16h and it would have taken many days -> do it again on GPU

### Iteration 3 - optimizing the first approach

Since it's ration, two worked better than iteration one, I thought the root cause may be that the first iteration did not have the proper context, for example, it lacked the connection to Wittmann Tours.
Therefore, I tried again to train on pretty much the raw data, but this also turned out to be incorrect.

```bash
./finetune \
        --model-base ./llama-2-7b-chat.Q4_K_M.gguf \
        --lora-out lora-llama-wittmann-tours-ITERATION.bin \
        --train-data "all-wittmann-tours-blog-posts3.txt" \
        --save-every 100 \
        --sample-start "<s>" \
        --threads 10 --adam-iter 620 --batch 4 --ctx 512 \
        --use-checkpointing
```

In [ ]:
# 3rd iteration
llm = Llama(model_path="llama-2-7b-chat.Q4_K_M.gguf",
            lora_path="./lora-llama-wittmann-tours-LATEST.bin",
            lora_base="models/Llama-2-7b-chat/ggml-model-f16.gguf",
            n_ctx=2048, verbose=False)

chat = Llama2_chat("Answer in a very concise and accurate way")
chat.prompt_llama2_stream("What do you know about wittmann-tours.de?")

### Iteration 4: Iteration 2 on Paperspace

Solve Login first